In [ ]:
# baseline 컬럼으로만 돌리는데
# 로그스케일링, 스탠다드 스케일링 적용
# 타겟값 wma14로 0값 대체

In [6]:
try_cnt = 9

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=19cd37258b23f2b6cb88c459587267ff9e5d6bfa312dbc0b0559932cb80252f6
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [15]:
!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 13.7 MB/s 


In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
import random
import os
import talib as tb

# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
tf.random.set_seed(19970119)
random.seed(19970119)
np.random.seed(19970119)

In [4]:
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

In [5]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):

    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [7]:
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

In [8]:
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=100)

    if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/check{try_cnt}') == False:
        os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/check{try_cnt}')

    filename = f'/content/drive/MyDrive/농산물예측/aT_data/check{try_cnt}/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

In [9]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.001, verbose = 1):


    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )


    # Train the model
    with tf.device('/device:GPU:0'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

In [10]:
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in tqdm(range(len(df) - seqence_length)):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

In [11]:

data_list = glob('/content/drive/MyDrive/농산물예측/aT_data/data/train/*.csv')
epoch = 1000
batch = 15
tr_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 '] # train 에서 사용하지 않는 열
ts_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)'] # test 에서 사용하지 않는 열
check_col = ['일자구분_중순', '일자구분_초순', '일자구분_하순','월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월', '월구분_3월', 
            '월구분_4월','월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월'] # 열 개수 맞추기

In [13]:
from sklearn.preprocessing import StandardScaler
def scaling_df(df):
  scaler = StandardScaler()
  
  
  tmp = df.drop('해당일자_전체평균가격(원)',axis=1).copy()
  scaler.fit(tmp)
  df_scaled = scaler.transform(tmp)
  df_scaled = pd.DataFrame(data=df_scaled, columns = tmp.columns)
  df_scaled['log_target'] = np.log1p(df['해당일자_전체평균가격(원)']).values 
  return df_scaled

In [18]:
for i in tqdm(data_list):
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)

    for j in df.columns:
        df[j] = df[j].replace({' ': np.nan})

    # 사용할 열 선택 및 index 설정
    df.drop(tr_del_list, axis=1, inplace=True)
    df.set_index('datadate', drop=True, inplace=True)

    # nan 처리
    df = df.fillna(0)


    # wma14로 0인값 채워주기
    df['wma14'] =  tb.WMA(df['해당일자_전체평균가격(원)'], 14)
    df['wma14'].fillna(method='bfill', inplace=True)
    df['해당일자_전체평균가격(원)'] = np.where(df['해당일자_전체평균가격(원)']==0, df['wma14'], df['해당일자_전체평균가격(원)'])
    df.drop('wma14', axis=1, inplace=True)

    # 스케일링
    df = scaling_df(df)


    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != 'log_target']], df['log_target']

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y

    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch,
                        batch)
    transformer.load_weights(f'/content/drive/MyDrive/농산물예측/aT_data/check{try_cnt}/transformer-{df_number}-{epoch}-{batch}.h5')

    if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/model{try_cnt}') == False:
        os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/model{try_cnt}')

    # 모델 저장
    transformer.save(f'/content/drive/MyDrive/농산물예측/aT_data/model{try_cnt}/transformer-{df_number}-{epoch}-{batch}.h5')

100%|██████████| 1447/1447 [00:00<00:00, 36035.42it/s]

100%|██████████| 1419/1419 [00:00<00:00, 34518.92it/s]


Epoch 1/1000
76/75 [==============================] - ETA: 0s - loss: 23.5542
Epoch 1: val_loss improved from inf to 1.06521, saving model to /content/drive/MyDrive/농산물예측/aT_data/check9/transformer-1-1000-15.h5
75/75 [==============================] - 16s 23ms/step - loss: 23.5542 - val_loss: 1.0652
Epoch 2/1000
73/75 [===========================>..] - ETA: 0s - loss: 5.3445
Epoch 2: val_loss did not improve from 1.06521
75/75 [==============================] - 1s 15ms/step - loss: 5.2667 - val_loss: 2.6600
Epoch 3/1000
73/75 [===========================>..] - ETA: 0s - loss: 4.6288
Epoch 3: val_loss did not improve from 1.06521
75/75 [==============================] - 1s 15ms/step - loss: 4.5978 - val_loss: 13.9084
Epoch 4/1000
73/75 [===========================>..] - ETA: 0s - loss: 10.7523
Epoch 4: val_loss did not improve from 1.06521
75/75 [==============================] - 1s 15ms/step - loss: 10.8644 - val_loss: 24.4518
Epoch 5/1000
73/75 [===========================>..] - ETA: 

100%|██████████| 1447/1447 [00:00<00:00, 37097.32it/s]

100%|██████████| 1419/1419 [00:00<00:00, 34141.89it/s]


Epoch 1/1000
73/75 [===========================>..] - ETA: 0s - loss: 29.8554
Epoch 1: val_loss improved from inf to 9.20205, saving model to /content/drive/MyDrive/농산물예측/aT_data/check9/transformer-0-1000-15.h5
75/75 [==============================] - 5s 21ms/step - loss: 29.1780 - val_loss: 9.2021
Epoch 2/1000
73/75 [===========================>..] - ETA: 0s - loss: 12.0871
Epoch 2: val_loss improved from 9.20205 to 3.09359, saving model to /content/drive/MyDrive/농산물예측/aT_data/check9/transformer-0-1000-15.h5
75/75 [==============================] - 1s 16ms/step - loss: 11.8644 - val_loss: 3.0936
Epoch 3/1000
73/75 [===========================>..] - ETA: 0s - loss: 8.2312
Epoch 3: val_loss did not improve from 3.09359
75/75 [==============================] - 1s 15ms/step - loss: 8.5234 - val_loss: 29.6957
Epoch 4/1000
73/75 [===========================>..] - ETA: 0s - loss: 12.8506
Epoch 4: val_loss improved from 3.09359 to 1.53098, saving model to /content/drive/MyDrive/농산물예측/aT_data/

100%|██████████| 1447/1447 [00:00<00:00, 35170.48it/s]

100%|██████████| 1419/1419 [00:00<00:00, 33015.21it/s]


...

100%|██████████| 1447/1447 [00:00<00:00, 30746.42it/s]

100%|██████████| 1419/1419 [00:00<00:00, 31368.58it/s]


Epoch 1/1000
73/75 [===========================>..] - ETA: 0s - loss: 28.1736
Epoch 1: val_loss improved from inf to 17.40027, saving model to /content/drive/MyDrive/농산물예측/aT_data/check9/transformer-32-1000-15.h5
75/75 [==============================] - 5s 21ms/step - loss: 27.6184 - val_loss: 17.4003
Epoch 2/1000
73/75 [===========================>..] - ETA: 0s - loss: 6.8887
Epoch 2: val_loss improved from 17.40027 to 1.00602, saving model to /content/drive/MyDrive/농산물예측/aT_data/check9/transformer-32-1000-15.h5
75/75 [==============================] - 1s 16ms/step - loss: 6.7981 - val_loss: 1.0060
Epoch 3/1000
73/75 [===========================>..] - ETA: 0s - loss: 4.2980
Epoch 3: val_loss improved from 1.00602 to 0.70501, saving model to /content/drive/MyDrive/농산물예측/aT_data/check9/transformer-32-1000-15.h5
75/75 [==============================] - 1s 16ms/step - loss: 4.2764 - val_loss: 0.7050
Epoch 4/1000
73/75 [===========================>..] - ETA: 0s - loss: 4.6864
Epoch 4: val_

100%|██████████| 1447/1447 [00:00<00:00, 34720.38it/s]

100%|██████████| 1419/1419 [00:00<00:00, 30204.25it/s]


Epoch 1/1000
73/75 [===========================>..] - ETA: 0s - loss: 29.8380
Epoch 1: val_loss improved from inf to 4.69194, saving model to /content/drive/MyDrive/농산물예측/aT_data/check9/transformer-27-1000-15.h5
75/75 [==============================] - 5s 21ms/step - loss: 29.1740 - val_loss: 4.6919
Epoch 2/1000
73/75 [===========================>..] - ETA: 0s - loss: 6.4954
Epoch 2: val_loss improved from 4.69194 to 0.83237, saving model to /content/drive/MyDrive/농산물예측/aT_data/check9/transformer-27-1000-15.h5
75/75 [==============================] - 1s 17ms/step - loss: 6.4595 - val_loss: 0.8324
Epoch 3/1000
73/75 [===========================>..] - ETA: 0s - loss: 7.3936
Epoch 3: val_loss did not improve from 0.83237
75/75 [==============================] - 1s 16ms/step - loss: 7.3909 - val_loss: 2.4549
Epoch 4/1000
73/75 [===========================>..] - ETA: 0s - loss: 11.4085
Epoch 4: val_loss did not improve from 0.83237
75/75 [==============================] - 1s 16ms/step - los

100%|██████████| 37/37 [2:59:21<00:00, 290.85s/it]


In [20]:
def test_scaling_df(df):
  scaler = StandardScaler()
  
  
 
  scaler.fit(df)
  df_scaled = scaler.transform(df)
  df_scaled = pd.DataFrame(data=df_scaled, columns = df.columns)
  return df_scaled

In [23]:
zero_csv = [0 for i in range(14)]  # 시점이 비어있는 데이터 0으로 채우기 위한 변수

for i in tqdm(range(10)):
    data_list = glob(f'/content/drive/MyDrive/농산물예측/aT_data/data/test/set_{i}/*.csv')

    for idx,j in enumerate(data_list):
        df = pd.read_csv(j)

        if len(df) == 0:
            df['zero_non'] = zero_csv
            df = df.fillna(0)
            df.drop('zero_non', axis=1, inplace=True)


        file_number = j.split('test_')[1].split('.')[0]

        # 사용할 열 선택, index 설정
        df.drop(ts_del_list, axis=1, inplace=True)
        df.set_index('datadate', drop=True, inplace=True)

        # train input 과 형상 맞추기
        add_col = [i for i in check_col if i not in df.columns]

        for a in add_col:
            df[a] = 0

        # ' ' -> nan 으로 변경
        for a in df.columns:
            df[a] = df[a].replace({' ': np.nan})

        # nan 처리
        df = df.fillna(0)


        # 어차피 타겟값 없애네
        # df['wma14'] =  tb.WMA(df['해당일자_전체평균가격(원)'], 14) # 이거 좀 미세한 숫자로 바꿨으면 좋겠는데..?
        # df['wma14'].fillna(method='bfill', inplace=True)
        # df['해당일자_전체평균가격(원)'] = np.where(df['해당일자_전체평균가격(원)']==0, df['wma14'], df['해당일자_전체평균가격(원)'])
        # df.drop('wma14', axis=1, inplace=True)


        df = test_scaling_df(df)



        # x_test  생성
        df_test = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))


        # model test
        if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}') == False:
            os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}')

        if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}/set_{i}') == False:
            os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}/set_{i}')

        # 해당하는 모델 불러오기
        model_test = tf.keras.models.load_model(f'/content/drive/MyDrive/농산물예측/aT_data/model{try_cnt}/transformer-{file_number}-{epoch}-{batch}.h5')
        pred = model_test.predict(df_test)


        pred = np.expm1(pred)


        # 결과 저장
        save_df = pd.DataFrame(pred).T
        save_df.to_csv(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}/set_{i}/predict_{file_number}.csv', index=False)

100%|██████████| 10/10 [10:38<00:00, 63.84s/it]


In [24]:
for k in tqdm(range(10)):

  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'/content/drive/MyDrive/농산물예측/aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [01:09<00:00,  6.92s/it]


In [25]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.1250,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.2800,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,3641.5125,1040.534700,2603.122300,1973.667500,2582.258300,5216.733000,2876.4102,2261.932400,4123.965300,1590.323200,...,5327.941000,531.895600,1115.096300,701.057400,3377.026000,3664.553700,6393.046400,2978.272700,2087.735800,1289.157300
2,3637.7314,1033.013500,2289.661000,1950.120400,2531.001200,5157.528300,2854.3760,2150.454000,4111.388000,1580.339100,...,5355.731000,517.807300,1141.363300,709.540340,3426.925000,3163.066200,6406.970700,2901.848600,1993.119000,1303.909800
3,3638.4880,999.570560,2037.982800,1962.017700,2529.042500,5131.970700,2844.6830,2184.165800,4196.723000,1556.098300,...,5296.737300,493.651340,1139.001600,697.442200,3413.707500,3007.779300,6386.074000,2887.028800,2096.036000,1295.373400
4,3632.6895,1029.396900,2633.171900,1987.174400,2579.836400,5186.863300,2881.3345,2393.534000,4105.372600,1601.869000,...,5286.909700,522.553470,1125.566500,705.789100,3415.068800,3061.229000,6505.349000,2877.020500,2021.846200,1310.553300
5,3646.6050,1032.880400,2724.589400,1982.619600,2589.663300,5251.366000,2728.8308,2363.700000,4094.430000,1623.228000,...,5248.311500,560.461100,1133.460700,716.587100,3387.922400,3238.878400,6472.898000,2856.282700,2060.547600,1308.312600
6,3651.0046,1045.017600,2587.392600,1996.766000,2609.596200,5275.213000,2790.7646,2474.047600,4098.888700,1617.238300,...,5280.000000,562.720700,1128.099600,718.729900,3419.228500,2628.270300,6566.468000,2968.683000,2105.368200,1317.373500
7,3653.0288,1043.710600,3004.765100,1979.020100,2612.358600,5347.369600,2839.7659,2554.347400,4086.848100,1634.188200,...,5308.770500,577.403930,1131.397200,711.358400,3444.456300,2675.212000,6483.656000,3080.555400,2047.296000,1310.830400
8,3656.1978,1030.733200,2499.380600,1963.114100,2610.521200,5350.135000,2793.6724,2473.293500,4041.373800,1591.833900,...,5381.396500,541.573100,1121.478500,708.961550,3421.284000,3894.358200,6437.501000,3035.304200,2002.579700,1315.523400
9,3645.9094,1022.980470,2697.874000,1947.377700,2514.395800,5278.510000,2739.7085,2364.810800,4034.998500,1576.595200,...,5371.381000,527.020200,1143.095200,705.233600,3481.830300,3759.185300,6340.196300,2959.003400,2049.167200,1315.795900


In [26]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']


for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값

    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1]- base_d)/base_d)  # t+1 ~ t+14 까지는 (t+n - t)/t 로 계산

    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean() - base_d)/base_d # t+22 ~ t+28은 np.mean(t+22 ~ t+28) - t / t

    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

In [27]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [28]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('/content/drive/MyDrive/농산물예측/aT_data/answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv(f'/content/drive/MyDrive/농산물예측/submit{try_cnt}.csv',index=False)